In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import os
import sys
import seaborn as sns
import pypsa
from itertools import product

sys.path.insert(0, os.getcwd() + "/../scripts")
import _helpers as h

plt.style.use(["../matplotlibrc","bmh"])
%matplotlib inline

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    peur_config = yaml.safe_load(stream)

In [ ]:
with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
def plot_violins(df, fidelity='high', fn=None):
    
    fig, axes = plt.subplots(1,4,figsize=(8,3), gridspec_kw={"width_ratios": [4,2,1,1], "hspace": 0.5, "wspace": 0.6})
    
    c = peur_config["plotting"]["tech_colors"]
    nn = peur_config["plotting"]["nice_names_n"]
    
    twkm_orig = 307 if fidelity == 'high' else 132
    
    kwargs = dict(inner='box', cut=0, scale='width', linewidth=1)
    
    sel = ["solar", "onwind", "offwind", "wind"]
    sns.violinplot(ax=axes[0], data=df[sel], **kwargs, palette=[c[s] for s in sel])
    axes[0].set_xticklabels([nn[i] for i in sel], rotation=15)
    
    sel = ["H2", "battery"]
    sns.violinplot(ax=axes[1], data=df[sel], **kwargs, palette=[c[s] for s in sel])
    axes[1].set_xticklabels([nn[i] for i in sel], rotation=15)
    
    sel = ["transmission"]
    sns.violinplot(ax=axes[2], data=df[sel], **kwargs, palette=[c[s] for s in sel])
    axes[2].set_xticklabels([nn[i] for i in sel], rotation=15)
    
    sel = ["tsc"]
    sns.violinplot(ax=axes[3], data=df[sel], **kwargs, palette=[c[s] for s in sel])
    axes[3].set_xticklabels([nn[i] for i in sel], rotation=15)
    
    axes[2].axhline(twkm_orig, linewidth=1, linestyle="--", color='gray', label='base')
    
    axes[2].set_ylim([0,900])
    axes[3].set_ylim([0,250])
    axes[1].set_ylim([0,300])
    axes[0].set_ylim([0,1500])
    
    axes[0].set_ylabel("GW")
    axes[1].set_ylabel("GW")
    axes[2].set_ylabel("TWkm")
    axes[3].set_ylabel("bn EUR p.a.")
    
    for ax in axes:
        ax.set_xlabel(None)
        ax.grid(False)
    
    axes[0].set_title(f"{len(df)} samples", fontsize=11)#, (-0.25,1200))
    
    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')

In [ ]:
fids = ["low", "high"]
for fid in fids:
    df = h.load_dataset(f"../results/dataset_{fid}.csv")
    plot_violins(df, fidelity=fid, fn=f"graphics/violins/violin-capacities-{fid}.pdf")

In [ ]:
uncertainties = config["uncertainties"]
distribution = h.NamedJ(uncertainties)
samples = distribution.sample(100000)

for fid in fids:
    surrogate = h.NamedPoly.from_txt(f"../results/pce/polynomial-{fid}-min-cost-0.0.txt")
    df = h.build_pce_prediction(surrogate, samples)
    plot_violins(df, fidelity=fid, fn=f"graphics/violins/violin-capacities-{fid}-prediction.pdf")